## **In this notebook we will be applying a flexible approach for loading and preprocessing the data that will allow for swiflty experimenting between different strategies.** 
## Particularly, this notebook covers:

* **Transfer Learning-Feature Extraction**
* **Data Augmentation** 
* **Exploiting meta features** 
* **Creating a model with multiple inputs** 
* **Saving the best model and making predictions**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import albumentations as A
import matplotlib.pyplot as plt
import tensorflow as tf 
import pandas as pd
import numpy as np 
import random 
import cv2 
import os

In [ ]:
# Load train and test data 
train_data = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
test_data = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')

# Store the paths 
train_path = '/kaggle/input/petfinder-pawpularity-score/train/'
test_path = '/kaggle/input/petfinder-pawpularity-score/test/'

In [ ]:
# Visualize some random images 
image_list = os.listdir('/kaggle/input/petfinder-pawpularity-score/train/')
IMG_SIZE = 512
random.seed(0)

for i in range(3):
    ax = plt.subplot(3, 3, i + 1)
    random_img = random.choice(image_list)
    x = random_img.split('.')[0]
    img = cv2.imread(os.path.join(train_path,random_img))
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    pawpularity = train_data.loc[train_data['Id'] == x, 'Pawpularity'].item()
    plt.title(f'{pawpularity}.')
    plt.imshow(img)
    plt.axis("off")
    plt.show()

## **Since we are loading the data from a directory and utilizing the meta-features (and labels) from a dataframe, we will create a flexible class that allows us to create both training and test data easily.**
## **Pixel-normalization can be applied, and image augmentation for the training data, although we need to be careful to not have any conflict with the meta-features (e.g. additional blur).**

In [ ]:
class DataProcessing(): 
    '''
    Class for creating training and test data. Combines feature extraction with meta-features.
    Returns featues from the images along with the corrsponding meta-data. 
    '''
    def __init__(self,train_df,train_path,test_df,test_path,IMG_SIZE):
        self.augmentation = None 
        self.normalization = None  
        self.model = None 
        
        self.train_path = train_path
        self.train_df = train_df
        self.test_path = test_path 
        self.test_df = test_df
        self.IMG_SIZE = IMG_SIZE 
        self.meta_features = ['Subject Focus','Eyes','Face','Near','Action','Accessory',\
                              'Group','Collage','Human','Occlusion','Info','Blur']
        
        self.transform = A.Compose([ A.CLAHE(),
                                    A.HorizontalFlip(),
                                    A.RandomRotate90(),
                                    A.Transpose(),
                                    A.ShiftScaleRotate(shift_limit=0.0425,
                                                       scale_limit=0.40, 
                                                       rotate_limit=25),
                                    A.HueSaturationValue()])
        
        self.normalize = A.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225],
                                    max_pixel_value=255.0,
                                    p=1.0)
        
    def feature_extractor(self):
        # Load base model 
        base_model = tf.keras.applications.EfficientNetB0(include_top=False)
        # Freeze the base model (so the pre-learned patterns remain)
        base_model.trainable = False
        # Inputs to the base model
        inputs = tf.keras.layers.Input(shape=(self.IMG_SIZE, self.IMG_SIZE, 3), name="input_layer")
        x = base_model(inputs)
        # Apply global average pooling to the outputs of the base model to aggregate the most important features
        outputs = tf.keras.layers.GlobalAveragePooling2D(name="average_pooling_layer")(x)
        model = tf.keras.Model(inputs, outputs)
        return model 
        
    def create_training_data(self,target_col,normalization=False,augmentation=False):
        X = []; y = []; meta = []
        # Initialize the model for feature extraction
        self.model = self.feature_extractor()
        train_img_ids = self.train_df['Id']
        meta_data = np.array(self.train_df[self.meta_features])
        for i,img in enumerate(train_img_ids):
            try:
                # Get the label of the current img 
                label = self.train_df.loc[self.train_df['Id'] == img, target_col].item()
                # Add the jpg to the relative path 
                img += '.jpg'
                # Read and resize the image 
                img = cv2.imread(os.path.join(self.train_path,img))
                img = cv2.resize(img, (self.IMG_SIZE,self.IMG_SIZE))
                # Apply image-augmentation if desired
                if self.augmentation:
                    img = self.transform(image=img)["image"]
                # Apply normalization if desired 
                if self.normalization:
                    img = self.normalize(image = img)['image']
                # Reshape the img
                img = img.reshape(1,IMG_SIZE,IMG_SIZE,3)
                # Extract features using transfer learning
                img = self.model.predict(img)
                # Get the meta features for the current img 
                meta_features = meta_data[i]
                X.append(img.squeeze())
                y.append(label)
                meta.append(meta_features)
            except Exception as e:
                print(e)     
        return X, y, meta                
    
    def create_test_data(self,normalization=False):
        X = [] ; meta = []
        self.model = self.feature_extractor()
        test_img_ids = self.test_df['Id']
        meta_data = np.array(self.test_df[self.meta_features])
        for i,img in enumerate(test_img_ids):
            try:
                img += '.jpg'
                img = cv2.imread(os.path.join(self.test_path,img))
                img = cv2.resize(img, (self.IMG_SIZE,self.IMG_SIZE))
                if self.normalization:
                    img = self.normalize(image = img)['image']
                img = img.reshape(1,IMG_SIZE,IMG_SIZE,3)
                img = self.model.predict(img)
                meta_features = meta_data[i]
                X.append(img.squeeze())
                meta.append(meta_features)
            except Exception as e:
                print(e)      
        return X, meta

In [ ]:
# Create the training data 
preprocessing = DataProcessing(train_data,train_path,test_data,test_path,IMG_SIZE=512)
X,y,meta_data = preprocessing.create_training_data(target_col='Pawpularity')

X = np.array(X,dtype=np.float32)
y = np.array(y,dtype=np.float32)
meta_data = np.array(meta_data)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_img, X_test_img, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train_meta, X_test_meta, y_train, y_test = train_test_split(meta_data,y, test_size=0.1, random_state=42)

In [ ]:
# Build a new model for classification using tensorflow's functional API
inputs_meta = tf.keras.layers.Input(shape= X_train_img[0].shape, name="input_meta_layer")
inputs_img = tf.keras.layers.Input(shape= X_train_meta[0].shape, name="input_img_layer")
# Concatenate both inputs
merged = tf.keras.layers.Concatenate()([inputs_meta, inputs_img])
x = tf.keras.layers.Dense(3000, kernel_initializer='normal',activation = tf.keras.layers.LeakyReLU())(merged)
x = tf.keras.layers.Dense(2000, kernel_initializer='normal',activation = tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(512, kernel_initializer='normal',activation = tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(256, kernel_initializer='normal',activation = tf.keras.layers.ReLU())(x)
x = tf.keras.layers.Dense(512, kernel_initializer='normal',activation = tf.keras.layers.ReLU())(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(1,kernel_initializer='normal')(x)
model = tf.keras.Model(inputs = [inputs_meta, inputs_img], outputs =outputs )
model.summary()
# Plot the model's architecture 
tf.keras.utils.plot_model(model,show_shapes=True,rankdir='TB',expand_nested=False,dpi=56)

In [ ]:
# Create a directory for saving the best model 
directory = '/kaggle/working/trained_model'
os.mkdir(directory)

In [ ]:
import tensorflow.keras.backend as K
EPOCHS = 450 

# Define the rmse for tracking model performance 
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Define the optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.00003, momentum=0.9)

# Compile the model 
model.compile(loss= rmse,
                  optimizer=optimizer, 
                  metrics=tf.keras.metrics.RootMeanSquaredError())

# Create checkpoints 
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/trained_model',
                                                monitor = 'val_root_mean_squared_error', 
                                                verbose = 0, 
                                                save_best_only = True)
# Save history 
history = model.fit([X_train_img,X_train_meta],y_train,validation_data = ([X_test_img,X_test_meta],y_test),
                        epochs=EPOCHS, verbose=0, callbacks=[checkpoint])


# Plot the model's loss curves 
pd.DataFrame(history.history).plot()
plt.ylabel("loss")
plt.xlabel("epochs")
plt.show()


In [ ]:
# Load the best model 
model = tf.keras.models.load_model('/kaggle/working/trained_model',compile=False)
# Utilize the best model for making predictions  
predictions = model.predict([X_test_img,X_test_meta]).squeeze().round()
# Get the error of the predictions with the true labels
rmse(predictions,y_test) 

In [ ]:
# Load test data and make predictions 
test_features,test_meta = preprocessing.create_test_data()
predictions = model.predict([np.array(test_features),np.array(test_meta)])
predictions = predictions.squeeze().squeeze()
test_data['Pawpularity'] = predictions 

In [ ]:
test_data = test_data[["Id", "Pawpularity"]]
test_data.to_csv("submission.csv", index=False)

In [ ]:
test_data